In [1]:
import os,cv2,torch,torchvision
import detectron2
from detectron2.evaluation import COCOEvaluator,inference_context,inference_on_dataset
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog,MetadataCatalog,build_detection_test_loader
from detectron2.engine import DefaultPredictor,DefaultTrainer
from detectron2.model_zoo import get_checkpoint_url,get_config_file
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import Visualizer,BoxMode
import numpy as np
import pandas as pd
from tqdm import tqdm
from xml.etree import ElementTree

In [2]:
imgs = []
x_mins = []
y_mins = []
y_maxs = []
x_maxs = []

In [3]:
for img in zip(os.listdir('./data/')):
    try:
        img = img[0]
        tree = ElementTree.parse(f'./data/{img}')
        root = tree.getroot()
        for box in root.findall('.//bndbox'):
            xmin = box.find('xmin').text
            xmax = box.find('xmax').text
            ymin = box.find('ymin').text
            ymax = box.find('ymax').text
        imgs.append('./data/' + img.replace('.xml','.jpg'))
        x_mins.append(int(xmin))
        x_maxs.append(int(xmax))
        y_mins.append(int(ymin))
        y_maxs.append(int(ymax))
    except Exception as e:
        pass

In [4]:
data = pd.DataFrame({'frame':imgs,'xmin':x_mins,'xmax':x_maxs,'ymax':y_maxs,'ymin':y_mins})

In [5]:
def load_data(data=data):
    dataset = []
    for idx in tqdm(range(len(data))):
        record = {}
        info = data.iloc[idx]
        xmin,ymin,xmax,ymax = info['xmin'],info['ymin'],info['xmax'],info['ymax']
        height,width = cv2.imread(info['frame']).shape[:2]
        record['file_name'] = info['frame']
        record['height'] = height
        record['width'] = width
        info['class_id'] = 0
        record['cateogry_id'] = 0
        objs = [{'bbox':[info['xmin'],info['ymin'],info['xmax'],info['ymax']],'bbox_mode':BoxMode.XYXY_ABS,'iscrowd':0,'category_id':0}]
        record['image_id'] = idx
        record['annotations'] = objs
        record['class_id'] = 0
        record['xmin'] = xmin
        record['ymin'] = ymin
        record['xmax'] = xmax
        record['ymax'] = ymax
        dataset.append(record)
    return dataset

In [6]:
DatasetCatalog.register('data',lambda : load_data())
MetadataCatalog.get('data').set(thing_classes=['PathHole'])
metadata = MetadataCatalog.get('data')

In [7]:
import wandb


In [8]:
import matplotlib.pyplot as plt


In [ ]:
torch.cuda.empty_cache()
wandb.init(sync_tensorboard=True,name='baseline')
model = 'COCO-Detection/faster_rcnn_R_50_C4_1x.yaml'
cfg = get_cfg()
cfg.merge_from_file(get_config_file(model))
cfg.DATASETS.TRAIN = ('data',)
cfg.DATASETS.TEST = ()
cfg.MODEL.WEIGHTS = get_checkpoint_url(model)
cfg.SOLVER.MAX_ITER = 2500 + 1250
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.STEPS = []
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(['PathHole'])
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.475
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
img = cv2.imread('./data/img-1.jpg')
predictor = DefaultPredictor(cfg)
preds = predictor(img)['instances'].to('cpu')
v = Visualizer(img[:,:,::-1],metadata=metadata,scale=1)
v = v.draw_instance_predictions(preds)
v = v.get_image()[:,:,::-1]
wandb.log({'img':wandb.Image(v)})
plt.figure(figsize=(10,7))
plt.imshow(v)
plt.show()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


[08/27 23:35:07 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
 

  0%|          | 0/665 [00:00<?, ?it/s]/home/indika/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/home/indika/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1724: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer_missing(indexer, value)
100%|██████████| 665/665 [00:02<00:00, 297.42it/s]

[08/27 23:35:09 d2.data.build]: Removed 0 images with no usable annotations. 665 images left.
[08/27 23:35:09 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|  PathHole  | 665          |
|            |              |
[08/27 23:35:09 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[08/27 23:35:09 d2.data.build]: Using training sampler TrainingSampler
[08/27 23:35:09 d2.data.common]: Serializing 665 elements to byte tensors and concatenating them all ...
[08/27 23:35:09 d2.data.common]: Serialized dataset takes 0.27 MiB



Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 2048) in the checkpoint but (2, 2048) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 2048) in the checkpoint but (4, 2048) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.


[08/27 23:35:10 d2.engine.train_loop]: Starting training from iteration 0


/home/indika/anaconda3/lib/python3.7/site-packages/detectron2/modeling/roi_heads/fast_rcnn.py:103: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  num_fg = fg_inds.nonzero().numel()


[08/27 23:35:17 d2.utils.events]:  eta: 0:19:44  iter: 19  total_loss: 1.091  loss_cls: 0.6581  loss_box_reg: 0.2556  loss_rpn_cls: 0.1701  loss_rpn_loc: 0.01687  time: 0.3361  data_time: 0.0137  lr: 4.9953e-06  max_mem: 2069M
[08/27 23:35:24 d2.utils.events]:  eta: 0:19:49  iter: 39  total_loss: 0.9372  loss_cls: 0.5843  loss_box_reg: 0.2001  loss_rpn_cls: 0.1319  loss_rpn_loc: 0.01246  time: 0.3440  data_time: 0.0026  lr: 9.9902e-06  max_mem: 2077M
[08/27 23:35:31 d2.utils.events]:  eta: 0:21:16  iter: 59  total_loss: 0.957  loss_cls: 0.4885  loss_box_reg: 0.2676  loss_rpn_cls: 0.1697  loss_rpn_loc: 0.03731  time: 0.3491  data_time: 0.0027  lr: 1.4985e-05  max_mem: 2081M
[08/27 23:35:38 d2.utils.events]:  eta: 0:21:16  iter: 79  total_loss: 0.9257  loss_cls: 0.3832  loss_box_reg: 0.2924  loss_rpn_cls: 0.1497  loss_rpn_loc: 0.01462  time: 0.3512  data_time: 0.0026  lr: 1.998e-05  max_mem: 2081M
[08/27 23:35:45 d2.utils.events]:  eta: 0:21:02  iter: 99  total_loss: 0.7653  loss_cls: 0.

In [ ]:
# from detectron2.checkpoint import DetectionCheckpointer
# checkpointer = DetectionCheckpointer(model, save_dir="output")
# checkpointer.save("model_999") 

In [ ]:
models = [
    'faster_rcnn_R_101_C4_3x.yaml',
    'faster_rcnn_R_101_DC5_3x.yaml',
    'faster_rcnn_R_101_FPN_3x.yaml',
    'faster_rcnn_R_50_C4_1x.yaml',
    'faster_rcnn_R_50_C4_3x.yaml',
    'faster_rcnn_R_50_DC5_1x.yaml',
    'faster_rcnn_R_50_DC5_3x.yaml',
    'faster_rcnn_R_50_FPN_1x.yaml',
    'faster_rcnn_R_50_FPN_3x.yaml',
    'faster_rcnn_X_101_32x8d_FPN_3x.yaml',
    'faster_rcnn_R_101_DC5_3x.yaml'
]

In [ ]:
BASE_LRS = [
    0.00025,
    0.00050,
    0.00075,
    0.0001,
    0.001,
    0.01,
    0.0002,
    0.0003,
    0.0004,
    0.0005,
    0.0006,
    0.0007,
    0.0008,
    0.0009,
]

In [ ]:
IMS_PER_BATCHS = [
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10
]

In [ ]:
BATCH_SIZE_PER_IMAGES = [
    121,
    122,
    123,
    124,
    125,
    126,
    127,
    129
]